In [1]:
import pandas as pd
import numpy as np
from pulp import LpProblem, LpMaximize, LpVariable
from pulp import LpInteger
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 替换为您系统中的中文字体
plt.rcParams['axes.unicode_minus'] = False
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sympy import symbols, exp, simplify
from scipy.optimize import linprog

# 读取CSV文件
file_path = 'modified_dataframe.csv'
df = pd.read_csv(file_path)

# 显示数据集的前几行
df.head()

,work_year,experience_level,employment_type,salary,remote_ratio,company_size,condensedsalary,safeZoneLow,safeZoneUp,safeZone
0,2023,1,3,175000,100,3,0.8750,0.5,1.0,0.5
1,2023,1,3,136000,0,1,-0.1000,-0.5,0.0,0.5
2,2023,1,3,141000,0,3,0.0250,0.0,0.5,0.5
3,2023,1,3,147100,0,3,0.1775,0.0,0.5,0.5
4,2023,1,3,130000,100,3,-0.2500,-0.5,0.0,0.5


In [2]:
def generate_dataframe():
    # 生成名为'value'的随机整数列
    df['value'] = np.random.randint(1, 51, size=len(df))

    # 生成均匀分布的'Uniform_privacy'列
    low_value = 1  # 分布的下限
    high_value = 4  # 分布的上限

    # 生成均匀分布数据
    df['Uniform_privacy'] = np.random.uniform(low=low_value, high=high_value, size=len(df))

    # 舍入到两位小数
    df['Uniform_privacy'] = df['Uniform_privacy'].round(2)

    return df

In [3]:
def pwp_mechanism_with_pldp(tau_low, tau_up, epsilon, d):
    # Step 1
    w = tau_up - tau_low
    h = w / 2 + tau_low
    t = d - h
    
    # Step 2
    a = np.random.rand()
    
    # Steps 3-7
    if a < np.exp(epsilon / 2) / (np.exp(epsilon / 2) + 1):
        # Steps 4-5
        l = (2 * t * np.exp(epsilon / 2) - w) / (2 * (np.exp(epsilon / 2) - 1))
        r = (2 * t * np.exp(epsilon / 2) + w) / (2 * (np.exp(epsilon / 2) - 1))
        et = np.random.uniform(l, r)
    else:
        # Steps 6
        C = w / 2 * (np.exp(epsilon / 2) + 1) / (np.exp(epsilon / 2) - 1)
        l = (2 * t * np.exp(epsilon / 2) - w) / (2 * (np.exp(epsilon / 2) - 1))
        r = (2 * t * np.exp(epsilon / 2) + w) / (2 * (np.exp(epsilon / 2) - 1))
        et = np.random.choice([np.random.uniform(-C, l), np.random.uniform(r, C)])
    
    # Step 8
    ed = et + h
    
    return ed

In [4]:
def laplace_mechanism(data, sensitivity, epsilon):
    shape = np.shape(data)
    
    # Generate Laplace noise
    noise = np.random.laplace(loc=0, scale=sensitivity / epsilon, size=shape)
    
    # Add noise to the data
    privatized_data = np.array(data) + noise
    
    return privatized_data

In [5]:
# 定义线性映射函数
def linear_map(value, from_low, from_high, to_low, to_high):
    return ((value - from_low) / (from_high - from_low)) * (to_high - to_low) + to_low

In [6]:
def calculate_value(theta_i, epsilon_i, w_i):
    # 根据给定的价值计算公式计算每个物品的价值
    return 4 * epsilon_i / (4 * np.exp(w_i / 4) * (epsilon_i + 1)) * theta_i

In [7]:
# 创建整数线性规划问题的函数
def solve_ilp_problem(B_prime):
    # 创建问题
    prob = LpProblem("最大化价值", LpMaximize)

    # 定义决策变量
    q = LpVariable.dicts("q", range(n), 0, 1, LpInteger)

    # 定义目标函数
    prob += sum(q[i] * values[i] for i in range(n)), "目标函数"

    # 定义约束条件
    prob += sum(q[i] * theta[i] for i in range(n)) <= B_prime, "约束条件"

    # 求解问题
    prob.solve()

    # 提取优化后的值
    optimized_q_values = [q[i].value() for i in range(n)]
    maximized_value = prob.objective.value()

    # 提取q值为1的索引
    selected_indices = [i for i, q_value in enumerate(optimized_q_values) if q_value == 1]
    
    # 根据q值为1的索引计算Age列和Uniform_privacy_age列的平均值
    average = np.mean(train_data['salary'].iloc[selected_indices])
    average_uniform = np.mean(train_data['Uniform_privacy_salary'].iloc[selected_indices])
    # 计算 RE
    RE = abs(average - average_uniform) / abs(average)

    model = LinearRegression()
    y_train = train_data['Uniform_privacy_salary'].iloc[selected_indices].values
    X_train = train_data[features].iloc[selected_indices].values
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 计算 RE
    RE_LINEAR = abs(y_pred - ages_in_test_data) / abs(y_pred)
    
    # 根据q值为1的索引计算Age列和Uniform_privacy_age列的平均值
    average = np.mean(train_data['salary'].iloc[selected_indices])
    average_uniform = np.mean(train_data['LAP_Uniform_privacy_salary'].iloc[selected_indices])
    # 计算 RE
    RE_LAP = abs(average - average_uniform) / abs(average)

    model = LinearRegression()
    y_train = train_data['LAP_Uniform_privacy_salary'].iloc[selected_indices].values
    X_train = train_data[features].iloc[selected_indices].values
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)    
    
    # 计算 RE
    RE_LINEAR_LAP = abs(y_pred - ages_in_test_data) / abs(y_pred)

    return RE, RE_LINEAR, RE_LAP, RE_LINEAR_LAP, maximized_value

In [8]:
def solve_greedy(values, theta, B_prime):
    # 将values从小到大排序
    sorted_indices = np.argsort(values)
    
    # 初始化变量
    current_sum = 0
    current_value = 0
    selected_indices = []

    # 遍历排序后的values
    for i in sorted_indices:
        # 检查是否添加当前值会超过约束条件
        if current_sum + theta[i] <= B_prime:
            # 添加当前值并更新当前总和
            selected_indices.append(i)
            current_sum += theta[i]
            current_value += values[i]

    # 计算所选索引对应的平均值
    average = np.mean(train_data['salary'].iloc[selected_indices])
    average_uniform = np.mean(train_data['Uniform_privacy_salary'].iloc[selected_indices])

    # 计算 RE
    RE = abs(average - average_uniform) / abs(average)

    model = LinearRegression()
    y_train = train_data['Uniform_privacy_salary'].iloc[selected_indices].values
    X_train = train_data[features].iloc[selected_indices].values
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 计算 RE
    RE_linear = abs(y_pred - ages_in_test_data) / abs(y_pred)

    return RE,current_value,RE_linear

In [9]:
def solve_lp_problem(B_prime):
    # 定义目标函数的系数（v_i * 𝜃_i^* / 50）
    c = [-values[i] * theta[i] / 50 for i in range(n)]

    # 定义不等式约束矩阵
    A = [[theta[i] / 50 for i in range(n)]]

    # 定义不等式约束右侧
    b = [B_prime]

    # 定义变量的取值范围
    bounds = [(0, 50) for _ in range(n)]

    # 求解线性规划问题
    result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

    # 提取优化后的值
    optimized_theta_values = result.x
    
    # 根据优化后的theta值计算q值和记录v_i值
    q_values = [1 if optimized_theta_values[i] >= theta[i] else 0 for i in range(n)]

    # 计算累加的v_i值
    maximized_value = sum(values[i] * q_values[i] for i in range(n))

    # 提取q值为1的索引
    selected_indices = [i for i, q_value in enumerate(q_values) if q_value == 1]

    # 根据q值为1的索引计算Age列和Uniform_privacy_age列的平均值
    average = np.mean(train_data['salary'].iloc[selected_indices])
    average_uniform = np.mean(train_data['Uniform_privacy_salary'].iloc[selected_indices])

    # 计算 RE
    RE = abs(average - average_uniform) / abs(average)

    model = LinearRegression()
    y_train = train_data['Uniform_privacy_salary'].iloc[selected_indices].values
    X_train = train_data[features].iloc[selected_indices].values
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 计算 RE
    RE_linear = abs(y_pred - ages_in_test_data) / abs(y_pred)

    return RE, maximized_value,RE_linear

In [10]:
# 创建整数线性规划问题的函数
def solve_ilp_problem_3(B_prime, privacy):
    # 创建问题
    prob = LpProblem("最大化价值", LpMaximize)

    # 定义决策变量
    q = LpVariable.dicts("q", range(n), 0, 1, LpInteger)

    # 定义目标函数
    prob += sum(q[i] * values[i] for i in range(n)), "目标函数"

    # 定义约束条件
    prob += sum(q[i] * theta[i] for i in range(n)) <= B_prime, "约束条件"
    
    # 添加新的约束条件 q_i * (epsilon_i - privacy) >= 0
    for i in range(n):
        prob += q[i] * (epsilon[i] - privacy) >= 0, f"New Constraint {i}"

    # 求解问题
    prob.solve()

    # 提取优化后的值
    optimized_q_values = [q[i].value() for i in range(n)]
    maximized_value = prob.objective.value()

    # 提取q值为1的索引
    selected_indices = [i for i, q_value in enumerate(optimized_q_values) if q_value == 1]
    
    # 根据q值为1的索引计算Age列和Uniform_privacy_age列的平均值
    average = np.mean(train_data['salary'].iloc[selected_indices])
    average_uniform = np.mean(train_data['Uniform_privacy_salary'].iloc[selected_indices])
    # 计算 RE
    RE = abs(average - average_uniform) / abs(average)
    
    model = LinearRegression()
    y_train = train_data['Uniform_privacy_salary'].iloc[selected_indices].values
    X_train = train_data[features].iloc[selected_indices].values
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # 计算 RE
    RE_linear = abs(y_pred - ages_in_test_data) / abs(y_pred)

    return RE, RE_linear

In [11]:
# 循环50次并保存结果到不同的Excel文件
for i in range(1, 51):
    generated_df = generate_dataframe()
    generated_df['Uniform_privacy_values'] = generated_df.apply(lambda row: pwp_mechanism_with_pldp(row['safeZoneLow'], row['safeZoneUp'], row['Uniform_privacy'], row['condensedsalary']), axis=1)
    generated_df['Uniform_privacy_salary'] = generated_df['Uniform_privacy_values'].apply(lambda x: linear_map(x, -1, 1, 100000, 180000))
    generated_df['LAP_Uniform_privacy_salary'] = generated_df.apply(lambda row: laplace_mechanism(row['salary'], 80000, row['Uniform_privacy']), axis=1)
    
    # 选择最后一个数据点作为测试集
    # 选择最后一个数据点作为测试集
    test_data = generated_df.iloc[-5:-4]
    features = ['work_year', 'experience_level', 'employment_type', 'remote_ratio', 'company_size']
    X_test = test_data[features].values
    # 剩余的数据作为训练集
    train_data = generated_df.iloc[:-5]
    ages_in_test_data = test_data['salary'].values
    
    # Get the data
    n = len(train_data)
    theta = train_data['value'].values
    fee = 10
    epsilon = train_data['Uniform_privacy'].values
    values = [calculate_value(theta_i, epsilon_i, 0.5) for theta_i, epsilon_i in zip(theta, epsilon)]
    FEE_1 = (4 * exp((1 - 1) / 4)) / (5 - 1) * fee 
    FEE_2 = (4 * exp((2 - 1) / 4)) / (5 - 2) * fee
    FEE_3 = (4 * exp((3 - 1) / 4)) / (5 - 3) * fee
    privacy_1 = 1
    privacy_2 = 2
    privacy_3 = 3
    
    # 增大B_prime的步长
    step_size = 1000
    B_values = list(range(1000, 20001, step_size))
    # 实验一
    RE_values_1 = []
    RE_LAP_values_1 = []
    RE_linear_values_1 = []
    RE_linear_LAP_values_1 = []
    # 实验二
    VALUE_PDT_2 = []
    VALUE_FQ_2 = []
    VALUE_SMQ_2 = []
    RE_FQ_2 = []
    RE_linear_FQ_2 = []
    RE_SMQ_2 = []
    RE_linear_SMQ_2 = []
    # 实验三
    step_size_3 = 500
    B_values_3 = list(range(2000, 10001, step_size_3))
    RE_privacy1_3 = []
    RE_privacy2_3 = []
    RE_privacy3_3 = []
    RE_linear_privacy1_3 = []
    RE_linear_privacy2_3 = []
    RE_linear_privacy3_3 = []
    
    for B_prime in B_values:
        RE, RE_LINEAR, RE_LAP, RE_LINEAR_LAP, VALUE = solve_ilp_problem(B_prime)
        RE_values_1.append(RE)
        RE_LAP_values_1.append(RE_LAP)
        RE_linear_values_1.append(RE_LINEAR)
        RE_linear_LAP_values_1.append(RE_LINEAR_LAP)
        VALUE_PDT_2.append(VALUE)
        print("实验一：B_prime:", B_prime)
        RE, VALUE, RE_LINEAR = solve_greedy(values, theta, B_prime)
        RE_FQ_2.append(RE)
        RE_linear_FQ_2.append(RE_LINEAR)
        VALUE_FQ_2.append(VALUE)
        RE, VALUE, RE_LINEAR = solve_lp_problem(B_prime)
        RE_SMQ_2.append(RE)
        RE_linear_SMQ_2.append(RE_LINEAR)
        VALUE_SMQ_2.append(VALUE)
        print("实验二：B_prime:", B_prime)
        
    for B_prime in B_values_3:
        B_1 = int(B_prime - FEE_1)
        RE_1, RE_linear_1 = solve_ilp_problem_3(B_1, privacy_1)
        RE_privacy1_3.append(RE_1)
        RE_linear_privacy1_3.append(RE_linear_1)
        B_2 = int(B_prime - FEE_2)
        RE_2, RE_linear_2 = solve_ilp_problem_3(B_2, privacy_2)
        RE_privacy2_3.append(RE_2)
        RE_linear_privacy2_3.append(RE_linear_2)
        B_3 = int(B_prime - FEE_3)
        RE_3, RE_linear_3 = solve_ilp_problem_3(B_3, privacy_3)
        RE_privacy3_3.append(RE_3)
        RE_linear_privacy3_3.append(RE_linear_3)
        print("实验三：B_prime:", B_prime)
    
    RE_linear_values_1 = np.array([val[0] for val in RE_linear_values_1])
    RE_linear_values_1 = RE_linear_values_1.tolist()
    RE_linear_LAP_values_1 = np.array([val[0] for val in RE_linear_LAP_values_1])
    RE_linear_LAP_values_1 = RE_linear_LAP_values_1.tolist()
    RE_linear_FQ_2 = np.array([val[0] for val in RE_linear_FQ_2])
    RE_linear_FQ_2 = RE_linear_FQ_2.tolist()
    RE_linear_SMQ_2 = np.array([val[0] for val in RE_linear_SMQ_2])
    RE_linear_SMQ_2 = RE_linear_SMQ_2.tolist()
    RE_linear_privacy1_3 = np.array([val[0] for val in RE_linear_privacy1_3])
    RE_linear_privacy1_3 = RE_linear_privacy1_3.tolist()
    RE_linear_privacy2_3 = np.array([val[0] for val in RE_linear_privacy2_3])
    RE_linear_privacy2_3 = RE_linear_privacy2_3.tolist()
    RE_linear_privacy3_3 = np.array([val[0] for val in RE_linear_privacy3_3])
    RE_linear_privacy3_3 = RE_linear_privacy3_3.tolist()
    
    data = {
        'RE_values_1': RE_values_1,
        'RE_LAP_values_1': RE_LAP_values_1,
        'RE_linear_values_1': RE_linear_values_1,
        'RE_linear_LAP_values_1': RE_linear_LAP_values_1,
        'VALUE_PDT_2': VALUE_PDT_2,
        'VALUE_FQ_2': VALUE_FQ_2,
        'VALUE_SMQ_2': VALUE_SMQ_2,
        'RE_FQ_2': RE_FQ_2,
        'RE_linear_FQ_2': RE_linear_FQ_2,
        'RE_SMQ_2': RE_SMQ_2,
        'RE_linear_SMQ_2': RE_linear_SMQ_2
    }

    data_3 = {
        'RE_privacy1_3': RE_privacy1_3,
        'RE_privacy2_3': RE_privacy2_3,
        'RE_privacy3_3': RE_privacy3_3,
        'RE_linear_privacy1_3': RE_linear_privacy1_3,
        'RE_linear_privacy2_3': RE_linear_privacy2_3,
        'RE_linear_privacy3_3': RE_linear_privacy3_3
    }

    result_12 = pd.DataFrame(data)
    result_3 = pd.DataFrame(data_3)
    
    # 保存到Excel文件
    result_12.to_excel(f'result_12_experiment_{i}.xlsx', index=False)
    result_3.to_excel(f'result_3_experiment_{i}.xlsx', index=False)

实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_

实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B

实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_pr

实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_pr

实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B

实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B_prime: 6500
实验三：B_prime: 7000
实验三：B_prime: 7500
实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B

实验三：B_prime: 8000
实验三：B_prime: 8500
实验三：B_prime: 9000
实验三：B_prime: 9500
实验三：B_prime: 10000
实验一：B_prime: 1000
实验二：B_prime: 1000
实验一：B_prime: 2000
实验二：B_prime: 2000
实验一：B_prime: 3000
实验二：B_prime: 3000
实验一：B_prime: 4000
实验二：B_prime: 4000
实验一：B_prime: 5000
实验二：B_prime: 5000
实验一：B_prime: 6000
实验二：B_prime: 6000
实验一：B_prime: 7000
实验二：B_prime: 7000
实验一：B_prime: 8000
实验二：B_prime: 8000
实验一：B_prime: 9000
实验二：B_prime: 9000
实验一：B_prime: 10000
实验二：B_prime: 10000
实验一：B_prime: 11000
实验二：B_prime: 11000
实验一：B_prime: 12000
实验二：B_prime: 12000
实验一：B_prime: 13000
实验二：B_prime: 13000
实验一：B_prime: 14000
实验二：B_prime: 14000
实验一：B_prime: 15000
实验二：B_prime: 15000
实验一：B_prime: 16000
实验二：B_prime: 16000
实验一：B_prime: 17000
实验二：B_prime: 17000
实验一：B_prime: 18000
实验二：B_prime: 18000
实验一：B_prime: 19000
实验二：B_prime: 19000
实验一：B_prime: 20000
实验二：B_prime: 20000
实验三：B_prime: 2000
实验三：B_prime: 2500
实验三：B_prime: 3000
实验三：B_prime: 3500
实验三：B_prime: 4000
实验三：B_prime: 4500
实验三：B_prime: 5000
实验三：B_prime: 5500
实验三：B_prime: 6000
实验三：B